In [ ]:
# 1. 단일 롯트 대표성 모델 분석 (Clopper-Pearson)

**연구 목표:** 제한된 샘플링 테스트 결과(n=11)를 바탕으로, 전체 비축물량의 신뢰도를 보수적으로 추정한다.

**분석 기법:**
- **Clopper-Pearson (정확 이항 검정 기반)**: 작은 샘플 사이즈에 적합한 신뢰구간 계산법.
- **단측 신뢰 하한 (One-sided Lower Confidence Bound)**: "최대"가 아닌 "최소" 신뢰도를 보장하는 데 초점을 맞춘 보수적 접근.

**핵심 가정:**
- **단일 모집단 가정**: 2015-2019년 생산된 모든 유도탄을 통계적으로 동일한 단일 모집단으로 간주한다.
- **기본 신뢰수준**: 90%

---

## 1. 환경 설정

분석에 필요한 라이브러리를 불러옵니다.

```python
import pandas as pd
from scipy.stats import beta
import matplotlib.pyplot as plt
import seaborn as sns
```

## 2. 분석 함수 정의

Clopper-Pearson 방법을 사용하여 신뢰도의 단측 하한(Lower Confidence Bound)을 계산하는 함수를 정의합니다. 신뢰수준의 기본값은 연구의 기본 가정인 90%로 설정합니다.

```python
def calculate_one_sided_lower_bound(
    num_samples: int, 
    num_failures: int, 
    confidence_level: float = 0.90
) -> dict:
    """
    Clopper-Pearson 방법을 사용하여 신뢰도의 단측 하한(Lower Confidence Bound)을 계산합니다.

    Args:
        num_samples (int): 총 샘플 수 (n).
        num_failures (int): 실패 수 (f).
        confidence_level (float): 계산하고자 하는 신뢰수준 (기본값: 0.90).

    Returns:
        dict: 분석 결과를 담은 딕셔너리.
    """
    num_successes = num_samples - num_failures
    alpha = 1 - confidence_level

    # 신뢰도 점 추정치 (Point Estimate of Reliability)
    reliability_point_estimate = num_successes / num_samples

    # 신뢰도 하한 (Lower Confidence Bound)
    lower_bound = beta.ppf(alpha, num_successes, num_failures + 1)
    
    # 실패가 0개일 때의 정확한 계산식 적용
    if num_successes == num_samples:
        lower_bound = alpha**(1 / num_samples)

    return {
        "technique": "Clopper-Pearson (One-sided)",
        "num_samples": num_samples,
        "num_failures": num_failures,
        "confidence_level": confidence_level,
        "reliability_point_estimate": reliability_point_estimate,
        "lower_confidence_bound": lower_bound,
    }
```

## 3. 시나리오별 분석 실행

11개 샘플 중 실패 개수가 0, 1, 2개인 세 가지 시나리오에 대해 분석을 수행합니다.

```python
# 분석할 시나리오 정의
scenarios = [
    {"failures": 0},
    {"failures": 1},
    {"failures": 2},
]

# 각 시나리오에 대해 분석 실행
results = []
for scenario in scenarios:
    result = calculate_one_sided_lower_bound(
        num_samples=11,
        num_failures=scenario["failures"]
    )
    results.append(result)

# 결과를 Pandas DataFrame으로 변환하여 가독성 높이기
results_df = pd.DataFrame(results)
```

## 4. 분석 결과 확인

계산된 신뢰도 점 추정치와 90% 신뢰도 하한을 확인합니다.

```python
print("--- 신뢰도 단측 하한 분석 결과 (90% 신뢰수준) ---")

for index, row in results_df.iterrows():
    print(f"\n[시나리오: 샘플 11개 중 {int(row['num_failures'])}개 실패]")
    print(f"  - 신뢰도 점 추정치: {row['reliability_point_estimate']:.3f} ({(row['reliability_point_estimate'])*100:.1f}%)")
    print(f"  - 90% 신뢰도 하한: {row['lower_confidence_bound']:.3f} ({(row['lower_confidence_bound'])*100:.1f}%)")
    print(f"    -> 해석: '전체 비축물량의 신뢰도는 90%의 신뢰수준으로 최소 {row['lower_confidence_bound']*100:.1f}% 이상이다.'")
```

## 5. 결과 시각화

각 시나리오별 신뢰도 점 추정치와 신뢰도 하한을 막대그래프로 시각화하여 직관적으로 비교합니다.

```python
# 시각화 스타일 설정
sns.set_theme(style="whitegrid")
plt.rcParams['font.family'] = 'Malgun Gothic' # Windows, 'AppleGothic' for Mac
plt.rcParams['axes.unicode_minus'] = False # 마이너스 폰트 깨짐 방지

# 그래프 그리기
fig, ax = plt.subplots(figsize=(10, 6))

# 시나리오별 막대 위치 설정
bar_width = 0.35
index = results_df.index

# 점 추정치 막대
bars1 = ax.bar(index - bar_width/2, results_df['reliability_point_estimate'], bar_width, 
               label='신뢰도 점 추정치 (R)', color='skyblue')

# 신뢰도 하한 막대
bars2 = ax.bar(index + bar_width/2, results_df['lower_confidence_bound'], bar_width, 
               label='90% 신뢰도 하한 (LCB)', color='salmon')

# 레이블 및 제목 설정
ax.set_xlabel('실패 개수', fontsize=12)
ax.set_ylabel('신뢰도', fontsize=12)
ax.set_title('실패 개수에 따른 신뢰도 추정치 변화 (Clopper-Pearson)', fontsize=15, pad=20)
ax.set_xticks(index)
ax.set_xticklabels([f"{f}개" for f in results_df['num_failures']])
ax.set_ylim(0, 1.1)
ax.legend()

# 막대 위에 값 표시
for bar in bars1 + bars2:
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2.0, yval + 0.01, f'{yval:.3f}', 
            ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()
```
